In [1]:
import sys,random,math
from collections import Counter
import numpy as np
import pandas as pd
np.random.seed(1)
random.seed(1)

In [2]:
data = pd.read_csv('IMDB.csv')
raw_reviews = list(data['review'])


In [3]:
for sent in raw_reviews:
    wordCnt = Counter()
    for word in sent.split(' '):
        wordCnt[word] -=1
vocab = list(set(map(lambda x:x[0],wordCnt.most_common())))



In [4]:
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
print(word2index)
concatinated = list()
input_dataset = list()
for sent in raw_reviews:
    sent_indices = list()
    for word in sent.split(' '):
        try:
            sent_indices.append(word2index[word])
            concatinated.append(word2index[word])
        except:
            ""
        
    input_dataset.append(sent_indices)
concatinated = np.array(concatinated)
random.shuffle(input_dataset)

{'say': 0, 'chance': 1, 'movies': 2, 'best': 3, 'characters': 4, 'and': 5, 'avoid': 6, 'far': 7, '/><br': 8, 'of': 9, 'muddled,': 10, 'in': 11, 'art,': 12, 'known': 13, 'movies.': 14, 'had': 15, 'channels': 16, 'worst': 17, 'who': 18, '-': 19, 'expect': 20, 'would': 21, "you'll": 22, 'rental,': 23, 'do': 24, 'Yosemite.<br': 25, 'see': 26, 'implausible': 27, 'left': 28, 'a': 29, 'McCoy': 30, 'only': 31, 'Trek': 32, 'as': 33, 'episodes.': 34, 'movie.': 35, 'just': 36, 'with': 37, 'interact': 38, 'but': 39, 'it': 40, 'me': 41, 'way': 42, 'fans': 43, 'even': 44, '/>I': 45, 'worth': 46, 'all': 47, 'watching,': 48, 'one': 49, 'nine': 50, 'however': 51, 'about': 52, 'be': 53, 'for': 54, 'that': 55, 'Fan': 56, 'watch': 57, 'renting': 58, 'Kirk,': 59, 'hardly': 60, 'goofy': 61, 'not': 62, 'scenes': 63, 'Unfortunately,': 64, 'Spock': 65, "can't": 66, 'good': 67, 'save': 68, 'to': 69, 'the': 70, 'movie': 71, 'needs': 72, 'Even': 73, 'movies,': 74, 'high': 75, 'far)': 76, 'Star': 77, 'by': 78, 'ca

In [5]:
alpha,iterations = 0.05,2
hidden_size,window,negative = (50,2,5)
weights_0_1 = (np.random.rand(len(vocab), hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab), hidden_size)*0

In [6]:
layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

In [7]:
def similar(target):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        diff = weights_0_1[index]-weights_0_1[target_index]
        sq_diff = diff**2
        scores[word] = -math.sqrt(sum(sq_diff))
    return scores.most_common(10)


In [8]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))


In [13]:
for rev_i,review in enumerate(input_dataset*iterations):
    for target_i in range(len(review)):
        target_samples = [review[target_i]]+list(concatinated[
            (np.random.rand(negative)*len(concatinated)).astype('int').tolist()
            ])
        left_context = review[max(0, target_i-window):target_i]
        right_context=review[target_i+1:min(len(review),target_i+window)]
        layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
        print(layer_1.shape)
        layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
        print(layer_2.shape)
        layer_2_delta = layer_2 - layer_2_target
        layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])
        weights_0_1[left_context+right_context] -= layer_1_delta * alpha
        weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1)*alpha
        break
    break

    if(rev_i % 250 == 0):
        sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
                                                * iterations)) + " " + str(similar('well')))
    sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
                                                * iterations)))


(50,)
(6,)


In [11]:
print(similar('well'))


[('avoid', nan), ('of', nan), ('movies', nan), ('/><br', nan), ('characters', nan), ('and', nan), ('say', nan), ('far', nan), ('best', nan), ('chance', nan)]


93